In [19]:
import torch
import torch.nn as nn
from torch.nn import functional as F
device='cuda' if torch.cuda.is_available() else ''
print(device)
block_size=8
batch_size=4
max_iters = 1000
learning_rate = 3e-4
eval_iters=250


cuda


In [20]:
with open('wizard_of_oz.txt','r',encoding='utf-8') as f:
    text=f.read()
chars = sorted(set(text))
print(chars)
vocab_size=len(chars)

['\n', ' ', '!', '"', '&', "'", '(', ')', '*', ',', '-', '.', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', '[', ']', '_', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '\ufeff']


In [21]:
string_to_int = {ch:i for i, ch in enumerate(chars)}
int_to_string = {i:ch for i, ch in enumerate(chars)}
encode = lambda s:[string_to_int[c] for c in s]
decode = lambda l:''.join([int_to_string[i] for i in l]) 

data = torch.tensor(encode(text), dtype=torch.long)

In [22]:
n = int(0.8*len(data))
train_data=data[:n]
val_data=data[n:]
 
def get_batch(split):
    data=train_data if split == 'train' else val_data
    ix = torch.randint(len(data)-block_size,(batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    x,y=x.to(device),y.to(device)
    return x, y

x,y = get_batch('train')
print('inputs:')
print(x)
print('targets:')
print(y)                                                                                                                                           

inputs:
tensor([[72,  1, 76, 58,  1, 76, 58, 71],
        [54, 72,  1, 73, 61, 58,  1, 71],
        [75, 62, 67, 60,  1, 67, 68,  1],
        [55, 58,  0, 58, 72, 69, 58, 56]], device='cuda:0')
targets:
tensor([[ 1, 76, 58,  1, 76, 58, 71, 58],
        [72,  1, 73, 61, 58,  1, 71, 58],
        [62, 67, 60,  1, 67, 68,  1, 71],
        [58,  0, 58, 72, 69, 58, 56, 62]], device='cuda:0')


In [27]:
@torch.no_grad()
def estimate_loss():
    out={}
    model.eval()
    for split in ['train','val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X,Y=get_batch(split)
            logits,loss=model(X,Y)
            losses[k]=loss.item()
        out[split]=losses.mean()
    model.train()
    return out

In [28]:
class BigramLanguageModel(nn.Module):
    def __init__(self,vocab_size):
        super().__init__()
        self.token_embedding_table=nn.Embedding(vocab_size,vocab_size)
    def forward(self,index,targets=None):
        logits=self.token_embedding_table(index)
        if targets is None:
            loss = None
        else:
            B,T,C=logits.shape
            logits=logits.view(B*T,C)
            targets=targets.view(B*T)
            loss=F.cross_entropy(logits,targets)
        return logits,loss
    def generate(self,index,max_new_tokens):
        #index is (B,T) array of indices of current context
        for _ in range(max_new_tokens):
            #get the predictions
            logits,loss = self.forward(index)
            #focus only on the last time step
            logits=logits[:,-1,:] #becomes (B,C)
            #apply softmax to get the probabilities
            probs=F.softmax(logits,dim=-1) #(B,C)
            #sample from the distribution
            index_next = torch.multinomial(probs,num_samples=1) #(B,1)
            #append sampled index to the running sequence
            index=torch.cat((index,index_next),dim=1)#(B,T+1)
        return index
model = BigramLanguageModel(vocab_size)
m=model.to(device)
context=torch.zeros((1,1),dtype=torch.long,device=device)
generated_chars=decode(m.generate(context,max_new_tokens=500)[0].tolist())
print(generated_chars)


izqQrHgE)sKaLm:FLyIpx(cRo)II::M:N;nf?t:N!QaE*Ev
wdEo﻿6d_S'ze2hGU-uS1GLia,yTmA5!,iXK9xgn[j.Gn5QZO!5prWOozV)uJ*Wx(4_0Tk3r[Osw8kNO1u90*6Cj[RN]O44sz;S_7QJAPk:EX8ovkNOyMq K7jL.
Eo3 -!oLt2k:jrHor"?H:I_H7-W*3kG1uPdeft-,3)R4zet9M﻿H4MN_JeAl﻿A)0Fv5vMI:v][GAP!P1J﻿qu5]Juquvg!Kv!NTG[Yn'pd[n'Qa-"rHO0u5rZoy﻿fAZak[S )Qj[JMuqH*Q70Em,iz(])KwUH0vHYYyF?!0TM2(]YO8﻿lbvG:m
H?6﻿5XkN:EVt9?1FOiPi(AM?AZy9znPcO9FT5p&mT8a﻿uszn'(Z-,g1YD,4y(v5I3vEoh9"qy3]J*Sk9O8fCD.C44_K﻿dTCV8eW7pyyRCVfd_7V0GsuOs)m*]J5﻿il5
j:mQS
jj1,R﻿_pyeyxk


In [36]:
#Create a Pytorch optimizer
optimizer = torch.optim.Adam(model.parameters(),lr=learning_rate)
for iter in range(max_iters):
    if iter%eval_iters==0:
        losses=estimate_loss()
        print(f"step: {iter}, train loss: {losses['train']:.3f}, val loss: {losses['val']:.3f}")

    #sample a batch of data
    xb,yb=get_batch('train')

    #evaluate the loss
    logits,loss =  model.forward(xb,yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()
print(loss.item())

step: 0, train loss: 3.866, val loss: 3.879
step: 250, train loss: 3.816, val loss: 3.855
step: 500, train loss: 3.765, val loss: 3.800
step: 750, train loss: 3.726, val loss: 3.780
3.9260165691375732
